# Table of Contents
* [Exploring fenaroli parsing](#Exploring-fenaroli-parsing)
	* [Loading plaintext generated by slate (on AWS instance)](#Loading-plaintext-generated-by-slate-%28on-AWS-instance%29)
	* [RE for flavor compounds](#RE-for-flavor-compounds)
	* [RE for ingredients](#RE-for-ingredients)


In [689]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, deque
%load_ext autoreload
%autoreload 2

In [3]:
import cPickle as pickle
import slate 
import re

# Exploring fenaroli parsing

## Loading plaintext generated by slate (on AWS instance)

In [296]:
with open('../data/doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)

In [295]:
cleaned_joined = joined.replace('.', ' ')

## RE for flavor compounds and ingredients

## Here I have the material I need to build the ingredient:compound mapping

In [808]:
def flav_ingr_mapper(raw_text):
    
    def check_for_header(new_comp, blobs, i, comp_pattern):
        for j in range(i,i+50):
            try:
                if new_comp[0] == re.findall(comp_pattern, blobs[j]):
                    return True
            except IndexError:
                pass
        return False
    
    flavor_dict = defaultdict(list)
    ignore_terms = ['EINECS No :']
    re.MULTILINE
    ingr_pattern = re.compile('Natural occurrence:(.+)')
    comp_pattern = re.compile('(.*[A-Z]{6,20}.*)')
    
    blobs = raw_text.split('\n\n')
    comp_q = deque([], maxlen=3)
    for i in range(len(blobs)):
        text = blobs[i].replace('\n', '')
        ing = re.findall(ingr_pattern, text)
        comp = re.findall(comp_pattern, text)
        if comp and comp[0] not in ignore_terms:
            if not check_for_header(comp, blobs, i, comp_pattern):
                comp_q.append(comp[0])
        if ing:
            try:
                flavor_dict[comp_q.pop().strip()] = ing[0].strip()
            except IndexError:
                pass
    return flavor_dict

In [809]:
flav_comp_dict = flav_ingr_mapper(cleaned_joined)

This is the compound - ingredient mapping I've been striving for

In [810]:
pd.Series(flavor_dict.values()).value_counts()
remove_terms = ['Not reported found in nature','n/a','Reported found in nature',
                'Data not found', 'Reported not found in nature', 'No data found ']

In [816]:
fcd = {k:v for k,v in flav_comp_dict.iteritems() if v not in remove_terms}

In [817]:
with open('./flavor_compund_map.pkl', 'w') as f:
    pickle.dump(fcd, f)